# Init

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Network Definition

In [2]:
class Net( nn.Module ):
    """ Create a multi layer network for classifying mnist """

    def __init__( self ):
        """ Set up the network """
        super().__init__()
        self.conv1 = nn.Conv2d( 1 ,  6 , 3 ) # kernel: 1 input image channel  ,  6 output channels, 3x3 square convolution
        self.conv2 = nn.Conv2d( 6 , 16 , 3 ) # kernel: 6 input sample channels, 16 output channels, 3x3 square convolution
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear( 16 * 6 * 6 , 120 )  # 6*6 from image dimension
        self.fc2 = nn.Linear( 120        ,  84 )
        self.fc3 = nn.Linear(  84        ,  10 )

    def forward( self , x ):
        """ Feed-forward (Classify) 
        You just have to define the forward function, and the backward function (where gradients are computed) is automatically defined for you using autograd. 
        You can use any of the Tensor operations in the forward function.
        """
        x = F.max_pool2d( F.relu( self.conv1( x ) ) , (2,2) ) # Max pooling over a (2, 2) window
        x = F.max_pool2d( F.relu( self.conv2( x ) ) , 2     ) # If the size is a square you can only specify a single number
        x = x.view( -1 , self.num_flat_features( x ) ) # Flatten
        x = F.relu( self.fc1( x ) )
        x = F.relu( self.fc2( x ) )
        x = self.fc3( x )
        return x

    def num_flat_features( self , x ):
        """ Get the number of features """
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [3]:
net = Net()
print( net )

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


# Test a dummy input

In [5]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.1161,  0.1031,  0.1137,  0.1401, -0.0887,  0.0926,  0.1225, -0.0719,
         -0.0472,  0.0322]], grad_fn=<AddmmBackward>)


# Reset

In [6]:
# Zero the gradient buffers of all parameters and backprops with random gradients:
net.zero_grad()
out.backward( torch.randn( 1 , 10 ) )

# Loss Function

In [7]:
output = net(input)
target = torch.randn( 10 )  # a dummy target, for example
target = target.view( 1 , -1 )  # make it the same shape as output
criterion = nn.MSELoss() # Mean Squared Error

loss = criterion( output , target )
print( loss )

tensor(0.5756, grad_fn=<MseLossBackward>)


In [8]:
# So, when we call loss.backward(), the whole graph is differentiated w.r.t. the loss
print( loss.grad_fn )  # MSELoss
print( loss.grad_fn.next_functions[0][0] )  # Linear
print( loss.grad_fn.next_functions[0][0].next_functions[0][0] )  # ReLU

# Backpropagation

In [9]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0142,  0.0078, -0.0067,  0.0184, -0.0058,  0.0044])


## Weight Update

In [10]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD( net.parameters() , lr = 0.01 )

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net( input )
loss = criterion( output , target )
loss.backward()
optimizer.step()    # Does the update